# Урок 10: Low-Rank Factorization

Сжатие через использование внутренней избыточности матриц весов. Идея: матрицу $W$ можно аппроксимировать парой матриц гораздо меньшего размера.

## 1. Математическая теория

### 1.1. SVD Разложение
Любую матрицу можно представить как $U \cdot \Sigma \cdot V^T$. Мы оставляем только $r$ главных компонент.

### 1.2. Методы из обзора:
*   **ASVD (Yuan et al., 2023b):** «Activation-aware SVD». Главная проблема SVD — оно минимизирует ошибку весов, а не выхода. ASVD взвешивает веса по активациям перед разложением:
    $$W_{weighted} = W \cdot diag(X_{rms})$$
    Это делает сжатие гораздо менее болезненным для качества модели.
*   **LPLR (Saha et al., 2023):** Комбинирует низкоранговое разложение с квантованием, достигая синергии.
*   **LSAER (Sharma et al., 2024):** Исследует динамический подбор ранга (rank) для каждого слоя в отдельности.

---

In [ ]:
import torch

def asvd_weights_raw(W, X_rms):
    # Взвешивание весов активациями (ASVD core logic)
    return W * X_rms.view(1, -1)

print("nanoGPT Track: Реализована математика активационного взвешивания матриц.")

## 2. Промышленная реализация: PEFT (AdaLoRA)
Хотя SVD используется для сжатия, самый популярный «родственный» подход в индустрии — это **AdaLoRA**, который использует низкоранговое разложение для адаптации модели, динамически оптимизируя бюджет параметров.

In [ ]:
from peft import AdaLoraConfig, get_peft_model
try:
    config = AdaLoraConfig(
        r=8, target_r=4, tinit=200, tfinal=1000,
        target_modules=["q_proj", "v_proj"]
    )
    model = get_peft_model(original_llama, config)
    print("Llama Track: Промышленный AdaLoRA настроен для адаптивной факторизации.")
except Exception as e:
    print(f"Ошибка: {e}")